In [99]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
from scipy.sparse import hstack

In [98]:
df=pd.read_csv("dataset-final.csv")

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2467 entries, 0 to 2466
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   MOVIE_NAME               2467 non-null   object 
 1   SYNOPSIS                 2317 non-null   object 
 2   YEAR                     2322 non-null   object 
 3   CAST                     2420 non-null   object 
 4   GENRES                   2442 non-null   object 
 5   PRODUCERS                1762 non-null   object 
 6   REVIEWS                  2467 non-null   object 
 7   DURATION_MIN             2142 non-null   float64
 8   RATING_SUR_5             2314 non-null   float64
 9   MAJORITY_SENTIMENT       2467 non-null   object 
 10  MOST_FREQUENT_SENTIMENT  2467 non-null   object 
 11  Rating_Category          2467 non-null   object 
dtypes: float64(2), object(10)
memory usage: 231.4+ KB


In [100]:
df

,MOVIE_NAME,SYNOPSIS,YEAR,CAST,GENRES,PRODUCERS,REVIEWS,DURATION_MIN,RATING_SUR_5,MAJORITY_SENTIMENT,MOST_FREQUENT_SENTIMENT,Rating_Category
0,wicked,A cul-de-sac in an oppressive suburb becomes a...,1998.0,"Julia Stiles, William R. Moses, Patrick Muldoo...",Thriller,Frank Beddor; Greg Steinberg,"['one unhinged viewing experience life', 'dont...",88.0,2.640,"['neutral', 'positive', 'neutral', 'neutral', ...",positive,Negative
1,gladiator-ii,Years after witnessing the death of the revere...,2024.0,"Paul Mescal, Denzel Washington, Pedro Pascal, ...",Drama; Action; Adventure,Michael A. Pruss; Winston Azzopardi; David Fra...,"['review may contain spoiler handle truth', 'c...",148.0,3.370,"['positive', 'neutral', 'neutral', 'negative',...",neutral,Neutral
2,moana-2,After receiving an unexpected call from her wa...,2024.0,"Auliʻi Cravalho, Dwayne Johnson, Hualālai Chun...",Comedy; Animation; Adventure; Family,Christina Chen; Yvett Merino Flores,"['water looked worse original', 'schaffrillas ...",100.0,2.870,"['negative', 'positive', 'positive', 'neutral'...",positive,Negative
3,the-substance,A fading celebrity decides to use a black mark...,2024.0,"Demi Moore, Margaret Qualley, Dennis Quaid, Ed...",Horror; Science Fiction,Coralie Fargeat; Eric Fellner; Tim Bevan,"['mama girl inside', 'review may contain spoil...",141.0,3.850,"['neutral', 'positive', 'positive', 'neutral',...",positive,Positive
4,our-little-secret,After discovering their significant others are...,2024.0,"Lindsay Lohan, Ian Harding, Kristin Chenoweth,...",Drama; Romance; Comedy,Mike Elliott; Lisa Gooding,['dating much younger woman close enough welco...,99.0,2.360,"['positive', 'negative', 'neutral', 'positive'...",positive,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...
2462,Along Came a Spider,A psychological suspense thriller adapted from...,2001,"Lee Tamahori (Director), Morgan Freeman (Alex ...",Mystery & Thriller,NaN,['nell minow blood violence murder good freema...,NaN,2.125,['positive'],positive,Negative
2463,Bad Behaviour,The film tells the story of Lucy (Jennifer Con...,2024,"Alice Englert (Director), Jennifer Connelly (L...",Comedy; Drama,NaN,['sheila omalley englert doesnt wrestle materi...,107.0,1.675,['negative'],negative,Negative
2464,Cabin Fever,A large family goes to the mountains to spend ...,2000,"Mona J. Hoel (Director), Svein Scharffenberg (...",Comedy; Drama,NaN,['jason gorber eli roth name watch guy created...,102.0,0.300,['positive'],positive,Negative
2465,Runaway Bride,"Having already left three grooms at the altar,...",1999,"Garry Marshall (Director), Julia Roberts (Magg...",Romance; Comedy,NaN,['david ansen runaway bride feel produce halfh...,NaN,2.450,['positive'],positive,Negative


In [92]:
# Compter le nombre de valeurs manquantes dans la colonne Rating_Category
missing_count = df["Rating_Category"].isna().sum()

print(f"Nombre de valeurs manquantes dans Rating_Category : {missing_count}")


Nombre de valeurs manquantes dans Rating_Category : 0


In [93]:
df.fillna("", inplace=True)

C:\Users\Aycha\AppData\Local\Temp\ipykernel_32172\2065217620.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("", inplace=True)


In [94]:
import numpy as np
import pandas as pd

# Fonction pour convertir les durées en minutes
def convert_duration_to_minutes(duration):
    if isinstance(duration, str):
        # Si la durée est exprimée en heures ('h')
        if 'h' in duration:
            hours = int(duration.replace('h', ''))  # Extraire le nombre d'heures
            return hours * 60  # Convertir en minutes
        # Si la durée est exprimée en minutes ('m')
        elif 'm' in duration:
            return int(duration.replace('m', ''))  # Extraire et retourner les minutes
    return 0  # Si la valeur est mal formatée, on retourne 0

# Appliquer la conversion de la colonne DURATION_MIN
df['DURATION_MIN'] = df['DURATION_MIN'].apply(convert_duration_to_minutes)

# Convertir YEAR en valeurs numériques
df['YEAR'] = pd.to_numeric(df['YEAR'], errors='coerce')

# Vérifier les valeurs
print(df)

# Conversion en type numérique
X_numeric = df[["DURATION_MIN", "YEAR"]].values
X_numeric = np.nan_to_num(X_numeric, nan=0.0)  # Remplacer NaN par 0
X_numeric = X_numeric.astype(np.float64)  # Assurez-vous du type numérique


               MOVIE_NAME                                           SYNOPSIS  \
0                  wicked  A cul-de-sac in an oppressive suburb becomes a...   
1            gladiator-ii  Years after witnessing the death of the revere...   
2                 moana-2  After receiving an unexpected call from her wa...   
3           the-substance  A fading celebrity decides to use a black mark...   
4       our-little-secret  After discovering their significant others are...   
...                   ...                                                ...   
2462  Along Came a Spider  A psychological suspense thriller adapted from...   
2463        Bad Behaviour  The film tells the story of Lucy (Jennifer Con...   
2464          Cabin Fever  A large family goes to the mountains to spend ...   
2465        Runaway Bride  Having already left three grooms at the altar,...   
2466           Anesthesia  The lives of a self-destructive student (Krist...   

        YEAR                           

In [80]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from scipy.sparse import hstack
import numpy as np

# Remplacer les valeurs manquantes (si elles existent)
df.fillna("", inplace=True)

# Vectorisation séparée pour chaque colonne textuelle
vectorizers = {}
text_features = ["SYNOPSIS", "GENRES", "CAST", "PRODUCERS", "REVIEWS"]
text_vectors = []

for feature in text_features:
    vectorizer = TfidfVectorizer()
    vectorized = vectorizer.fit_transform(df[feature])
    vectorizers[feature] = vectorizer
    text_vectors.append(vectorized)

# Combinaison des colonnes vectorisées
X_text = hstack(text_vectors)
df['DURATION_MIN'] = df['DURATION_MIN'].replace('', 0)
df['YEAR'] = df['YEAR'].replace('', 0)
# Ajout des colonnes numériques
X_numeric = df[["DURATION_MIN", "YEAR"]].values
X_numeric = np.nan_to_num(X_numeric, nan=0.0)  # Remplace NaN par 0
X_numeric = X_numeric.astype(np.float64)  # Assurez-vous du type numérique
# Fusion des caractéristiques textuelles et numériques
from scipy.sparse import csr_matrix

X_combined = hstack([X_text, csr_matrix(X_numeric)])

# Mapper les catégories en entiers
df["Rating_Category"] = df["Rating_Category"].map({"Negative": 0, "Neutral": 1, "Positive": 2})

# Reprendre la division des données
X_train, X_test, y_train, y_test = train_test_split(X_combined, df["Rating_Category"], test_size=0.2, random_state=42)

# Entraîner le modèle
model = RandomForestClassifier(random_state=42)

model.fit(X_train, y_train)


# Évaluation
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


C:\Users\Aycha\AppData\Local\Temp\ipykernel_32172\775059786.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("", inplace=True)
C:\Users\Aycha\AppData\Local\Temp\ipykernel_32172\775059786.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['YEAR'] = df['YEAR'].replace('', 0)


              precision    recall  f1-score   support

           0       0.80      0.63      0.71       145
           1       1.00      0.12      0.22       120
           2       0.61      0.97      0.75       229

    accuracy                           0.66       494
   macro avg       0.80      0.57      0.56       494
weighted avg       0.76      0.66      0.61       494



In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from scipy.sparse import hstack, csr_matrix
import numpy as np

# Remplacer les valeurs manquantes
df.fillna("", inplace=True)

# Vectorisation TF-IDF pour les colonnes textuelles
text_features = ["SYNOPSIS", "GENRES", "CAST", "PRODUCERS", "REVIEWS"]
vectorizers = {feature: TfidfVectorizer() for feature in text_features}
text_vectors = [vectorizers[feature].fit_transform(df[feature]) for feature in text_features]
X_text = hstack(text_vectors)

# Prétraitement des colonnes numériques
df['DURATION_MIN'] = df['DURATION_MIN'].replace('', 0).astype(float)
df['YEAR'] = df['YEAR'].replace('', 0).astype(float)
X_numeric = csr_matrix(df[["DURATION_MIN", "YEAR"]].values)

# Fusion des caractéristiques textuelles et numériques
X_combined = hstack([X_text, X_numeric])

# Mapper les catégories en entiers
df["Rating_Category"] = df["Rating_Category"].map({"Negative": 0, "Neutral": 1, "Positive": 2})
y = df["Rating_Category"]

# Équilibrage des poids des classes
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Division des données
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Modèle avec ajustement des poids
model = RandomForestClassifier(random_state=42, class_weight=class_weights_dict)
model.fit(X_train, y_train)

# Évaluation
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


In [97]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from scipy.sparse import hstack, csr_matrix
import numpy as np
from xgboost import XGBClassifier

# Remplacer les valeurs manquantes
df.fillna("", inplace=True)

# Vectorisation TF-IDF pour les colonnes textuelles
text_features = ["SYNOPSIS", "GENRES", "CAST", "PRODUCERS", "REVIEWS"]
vectorizers = {feature: TfidfVectorizer() for feature in text_features}
text_vectors = [vectorizers[feature].fit_transform(df[feature]) for feature in text_features]
X_text = hstack(text_vectors)

# Prétraitement des colonnes numériques
df['DURATION_MIN'] = df['DURATION_MIN'].replace('', 0).astype(float)
df['YEAR'] = df['YEAR'].replace('', 0).astype(float)
X_numeric = csr_matrix(df[["DURATION_MIN", "YEAR"]].values)

# Fusion des caractéristiques textuelles et numériques
X_combined = hstack([X_text, X_numeric])

# Mapper les catégories en entiers
df["Rating_Category"] = df["Rating_Category"].map({"Negative": 0, "Neutral": 1, "Positive": 2})
y = df["Rating_Category"]

# Division des données
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Équilibrage des poids des classes pour XGBoost
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
scale_pos_weight = class_weights_dict[0]  # Utilisation pour la classe la moins représentée

# Modèle XGBoost
model = XGBClassifier(
    random_state=42,
    scale_pos_weight=scale_pos_weight,  # Gérer le déséquilibre
    eval_metric="mlogloss",            # Fonction de perte multi-classes
    use_label_encoder=False,
    objective="multi:softmax",         # Pour une classification multi-classes
    num_class=3                        # Nombre de classes
)

# Entraîner le modèle
model.fit(X_train, y_train)

# Évaluation
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


C:\Users\Aycha\AppData\Local\Temp\ipykernel_32172\688094800.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("", inplace=True)


ValueError: classes should include all valid labels that can be in y